# Introduction

Python의 Model I/O에 대해 학습함

## FewShotPromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [4]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [16]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'question': 'What do you know about Italy?', 'answer': '\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'question': 'What do you know about Greece?', 'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}'), suffix='Human: What do you know about {country}?')

In [6]:

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella and Tapas
        Currency: Euro

AIMessage(content='AI: I know this:\n        Capital: Madrid\n        Language: Spanish\n        Food: Paella and Tapas\n        Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--c62e0a9d-6cc5-4cbb-bea1-bdc1931fd1de-0')

## FewShotChatMessagePromptTemplate

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [8]:
example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}"),
  ]
)

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you kjnow about {country}?")
  ]
)

In [9]:
chain= final_prompt | chat

chain.invoke(
  {
    "country" : "Korea"
  }
)


        I know this:
        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).
        Languages: Korean
        Food: Kimchi and Bibimbap
        Currency: North Korean Won (North) and South Korean Won (South)

AIMessage(content='\n        I know this:\n        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).\n        Languages: Korean\n        Food: Kimchi and Bibimbap\n        Currency: North Korean Won (North) and South Korean Won (South)', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--f6b328e7-149c-468f-b945-3b0c0e293d8d-0')

## LengthBasedExampleSelector

In [17]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [18]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)

In [20]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Brazil")

In [21]:
chain = prompt | chat
chain.invoke({"country": "Spain"})

AI: Here is what I know about Spain:

- **Capital**: Madrid
- **Language**: Spanish (Castilian)
- **Food**: Tapas, paella, and jamón ibérico
- **Currency**: Euro
- **Culture**: Known for its rich history, art, flamenco music and dance, and festivals like La Tomatina and Running of the Bulls.
- **Geography**: Located in southwestern Europe, bordered by France to the northeast and Portugal to the west, with coastlines along the Atlantic Ocean and the Mediterranean Sea.

AIMessage(content='AI: Here is what I know about Spain:\n\n- **Capital**: Madrid\n- **Language**: Spanish (Castilian)\n- **Food**: Tapas, paella, and jamón ibérico\n- **Currency**: Euro\n- **Culture**: Known for its rich history, art, flamenco music and dance, and festivals like La Tomatina and Running of the Bulls.\n- **Geography**: Located in southwestern Europe, bordered by France to the northeast and Portugal to the west, with coastlines along the Atlantic Ocean and the Mediterranean Sea.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--8248c1db-680d-4a4f-b7fd-86df64d7c617-0')

## RandomBasedExampleSelector

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [11]:
class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples
  
  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice

    return [choice(self.examples)]

In [12]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
  examples=examples
)

In [24]:
prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## Serialization and Composition

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

# prompt.format(country="Germmany")

'What is the capital of Germmany'

In [28]:
chain = prompt | chat

chain.invoke({
    "country" : "Germany"
})

The capital of Germany is Berlin.

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--0e3c6e7a-27bf-4d1e-99be-812ed17636d4-0')

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

In [22]:
intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
  """
)

example = PromptTemplate.from_template(
  """
  this is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
  """
)

start= PromptTemplate.from_template(
  """
  Strat now!

  Human: {question}
  You:
  """
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
  """
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--6ff77aeb-e198-41e9-9c25-22eaeee7ac1a-0')

## Caching

In [32]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

chat.predict("What is the capital of Japan?")


[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan?"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 14,
                "total_tokens": 21,
                "completion_tokens_details": {
            

'The capital of Japan is Tokyo.'

In [31]:
chat.predict("What is the capital of Japan?")

'The capital of Japan is Tokyo.'

## Serialization

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False)

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1
)

with get_openai_callback() as usage:
    chat.predict("What is the recipt for soju")
    chat.predict("What is the recipt for bread")
    
print(usage)
print(usage.total_cost)
print(usage.total_tokens)

Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')
Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')


Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
0.0
0
